In [ ]:
#libera memória GPU
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

# Imports

In [ ]:
from collections import OrderedDict
import os
from skimage.util import random_noise
import numpy as np
from tqdm import tqdm
from random import shuffle
import torch.nn.functional as F
import torch
from numpy import linalg as LA
import random
import time
import torch
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import torchvision
from torchvision import models
from torch.optim.lr_scheduler import StepLR
from matplotlib import pyplot as plt
import math
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler, BatchSampler, RandomSampler
import torch.nn as nn
from torchvision import transforms, datasets, models
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from torchsummary import summary
from random import randint
from __future__ import print_function
from __future__ import division
import copy
import csv
import sklearn
import matplotlib as mpl
from scipy.stats import rankdata
RS = 23

torch.backends.cudnn.deterministic = True
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)

# Preparação dos dados

O dataset deve estar em formato zip no google drive

In [ ]:
# monta o google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/My\ Drive/chest_xray.zip

In [ ]:
!rm -rf /content/chest_xray-bal

# Funções

In [ ]:
def prepararDados(input_size):
  datadir = '/content/chest_xray'
  traindir = datadir + '/train/'
  validdir = datadir + '/val/'
  testdir = datadir + '/test/'
 
  batch_size = 32

  image_transforms = {
    'train': transforms.Compose([    
        transforms.Resize((input_size,input_size)),        
        transforms.RandomHorizontalFlip(), 
        transforms.RandomResizedCrop((input_size,input_size)),       
        transforms.ToTensor(), 
      ]),

    'test': transforms.Compose([
        transforms.Resize((input_size,input_size)),      
        transforms.ToTensor(),
            ]),

    'val': transforms.Compose([
        transforms.Resize((input_size,input_size)),       
        transforms.ToTensor(),
            ])
  }
  
  # Datasets
  data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
    'val':
    datasets.ImageFolder(root=testdir, transform=image_transforms['val']),
    'test':
    datasets.ImageFolder(root=testdir, transform=image_transforms['test'])
  } 


  dataloaders = {
      'train': DataLoader(data['train'], shuffle=True, batch_size=batch_size, num_workers=0),
      'val': DataLoader(data['val'], batch_size=batch_size, shuffle=True),
      'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
  }
  return dataloaders

def create_tensor(shape = [2,2,2]):
    ndim = len(shape)
    view_shape = [1,] * ndim
    res = 0.
    for d in range(ndim):
        dim_size = shape[d]
        t = torch.arange(dim_size)
        view_shape[d] = dim_size
        t = t.view(*view_shape).expand(*shape)
        view_shape[d] = 1
        res += t

    return res

def testeDoModelo(model, data_loader):
  if (torch.cuda.is_available()):
    model = model.cuda()
  model.eval()
  running_loss = 0.0
  running_correct = 0
  tot_predictions = []
  labels = []
  for batch in data_loader:
    data , target = batch
    if torch.cuda.is_available():
      data,target = data.cuda(),target.cuda()
    
    output = model(data)
    loss = F.cross_entropy(output,target)
    running_loss += loss.item()
    preds = output.data.max(dim=1,keepdim=True)[1]
    running_correct += preds.eq(target.data.view_as(preds)).cpu().sum().item()
    
    tot_predictions.extend(preds.data.cpu().numpy())
    labels.extend(target.data.cpu().numpy())
  loss = running_loss/len(data_loader.dataset)
  accuracy = 100. * running_correct/len(data_loader.dataset)
  print(f'Loss is {loss:{5}.{2}} Accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{6}.{4}}%\n')
  return tot_predictions, labels


def testeAgrupamento(resnet, vgg, densenet, alexnet, googlenet, data_loader):
  if (torch.cuda.is_available()):
    resnet = resnet.cuda()
    vgg = vgg.cuda()
    densenet = densenet.cuda()
    alexnet = alexnet.cuda()
    googlenet = googlenet.cuda()
  
  resnet.eval()
  densenet.eval()
  vgg.eval()
  alexnet.eval()
  googlenet.eval()

  running_loss = 0.0
  running_corrects = 0
  tot_predictions = []
  labels = []
  for batch in data_loader:
    data , target = batch
    if torch.cuda.is_available():
      data,target = data.cuda(),target.cuda()
    
    output_resnet = resnet(data)
    output_densenet = densenet(data)
    output_alexnet = alexnet(data)
    output_googlenet = googlenet(data)
    output_vgg = vgg(data)
    
    # #resize for inception    
    # data = data.cpu()   
    # data_resized = create_tensor(shape=[len(data), 3, 299, 299])
    # for i in range(len(data)):      
    #   img_PIL = transforms.ToPILImage()(data[i])
    #   img_PIL = torchvision.transforms.Resize([299,299])(img_PIL)
    #   data_resized[i] = torchvision.transforms.ToTensor()(img_PIL) 

    # data_resized = data_resized.cuda()
    # output_inception = inception(data_resized)

    preds_resnet = output_resnet.data.max(dim=1,keepdim=True)[1]
    preds_densenet = output_densenet.data.max(dim=1,keepdim=True)[1]
    preds_vgg = output_vgg.data.max(dim=1,keepdim=True)[1]
    preds_googlenet = output_googlenet.data.max(dim=1,keepdim=True)[1]
    preds_alexnet = output_alexnet.data.max(dim=1,keepdim=True)[1]

    final_preds = []
    for i in range(len(preds_resnet)):
      item_pred = [preds_resnet[i].item(), preds_densenet[i].item(), preds_vgg[i].item(), preds_googlenet[i].item(), preds_alexnet[i].item()]
      soma = sum(item_pred)
      if soma >= 3:
        final_preds.append(1)
      else: 
        final_preds.append(0)

    for i in range(len(target)):
      if target[i] == final_preds[i]: 
        running_corrects += 1
    
    tot_predictions.extend(final_preds)
    labels.extend(target.data.cpu().numpy())
    

  accuracy = 100. * running_corrects/len(data_loader.dataset)
  print(f'Accuracy is {running_corrects}/{len(data_loader.dataset)}{accuracy:{6}.{4}}%\n')
  return tot_predictions, labels
  

def testeAgrupamentoMediaLogits(resnet, vgg, densenet, alexnet, googlenet, data_loader):
  if (torch.cuda.is_available()):
    resnet = resnet.cuda()
    vgg = vgg.cuda()
    densenet = densenet.cuda()
    alexnet = alexnet.cuda()
    googlenet = googlenet.cuda()
  
  resnet.eval()
  densenet.eval()
  vgg.eval()
  alexnet.eval()
  googlenet.eval()

  running_loss = 0.0
  running_corrects = 0
  tot_predictions = []
  labels = []
  for batch in data_loader:
    data , target = batch
    if torch.cuda.is_available():
      data,target = data.cuda(),target.cuda()
    
    output_resnet = resnet(data)
    output_densenet = densenet(data)
    output_alexnet = alexnet(data)
    output_googlenet = googlenet(data)
    output_vgg = vgg(data)

    outputs = (output_resnet+output_densenet+output_alexnet+output_googlenet+output_vgg)/5
      
    _, preds = torch.max(outputs, 1)
    

    for i in range(len(target)):
      if target[i] == preds[i]: 
        running_corrects += 1

    final_preds = []
    for i in range(len(preds)):
      final_preds.append(preds[i].item())
    
    tot_predictions.extend(final_preds)
    labels.extend(target.data.cpu().numpy())
  

  accuracy = 100. * running_corrects/len(data_loader.dataset)
  print(f'Accuracy is {running_corrects}/{len(data_loader.dataset)}{accuracy:{6}.{4}}%\n')
  return tot_predictions, labels


def metricasModelo(model, data_loader):
  if (torch.cuda.is_available()):
    model = model.cuda()
    
  print('========= Métricas do modelo ============\n')
  y_pred, y_test = testeDoModelo(model, data_loader)
  target_names = ['normal', 'pneumonia']
  print(classification_report(y_test, y_pred, target_names=target_names, digits=3))
  print('\n========= Confusion Matrix ============\n')
  confmat = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[0,1])
  fig, ax = plt.subplots(figsize=(3,3))
  ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
  for i in range(confmat.shape[0]):
      for j in range(confmat.shape[1]):
          ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
  plt.xlabel('predicted label')
  plt.ylabel('true label')
  plt.tight_layout()
  plt.savefig('confusion_matrix.png', dpi=300)
  plt.show()

def carregar_resnet():
  resnet, input_size = initialize_model('resnet', 2, True, use_pretrained=True)
  resnet.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/Resnet18/resnet.acc'))
  return resnet, input_size

def carregar_densenet():
  densenet, input_size = initialize_model('densenet', 2, True, use_pretrained=True)
  densenet.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/Densenet121/densenet.acc'))
  return densenet, input_size

def carregar_alexnet():
  alexnet, input_size = initialize_model('alexnet', 2, True, use_pretrained=True)
  alexnet.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/AlexNet/alexnet.acc'))
  return alexnet, input_size

def carregar_googlenet():
  googlenet, input_size = initialize_model('googlenet', 2, True, use_pretrained=True)
  googlenet.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/GoogLeNet/googlenet.acc'))
  return googlenet, input_size

def carregar_inception():
  inception, input_size = initialize_model('inception', 2, True, use_pretrained=True)
  inception.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/Inception V3/inception.acc'))
  return inception, input_size

def carregar_vgg():
  vgg = models.vgg19_bn(pretrained=True)
  num_ftrs = vgg.classifier[6].in_features
  vgg.classifier[6] = nn.Linear(num_ftrs, 2)
  vgg.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/VGG/vgg.acc'))
  return vgg, 224

def carregar_cnn(model_save_name):
    model = Network()    
    model.load_state_dict(torch.load(model_save_name))
    return model

def carregar_ensemble():
  #load the models
  resnet, input_size = carregar_resnet()
  vgg, input_size = carregar_vgg()
  densenet, input_size = carregar_densenet()
  alexnet, input_size = carregar_alexnet()
  googlenet, input_size = carregar_googlenet()
  ensemble = {'resnet': resnet, 'vgg': vgg, 'densenet': densenet, 'alexnet': alexnet, 'googlenet': googlenet}
  return ensemble

def salvar_modelo(model, model_save_name):
    torch.save(model.state_dict(), model_save_name)  

def plotar(train_loss, train_acc, val_loss, val_acc):
  plt.title('Accuracy over epochs')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.plot(train_acc, label='train')
  plt.plot(val_acc, label='val')
  plt.grid(True, which='both', axis='both')  
  plt.tight_layout()
  plt.legend()
  plt.show()
  plt.title('Loss over epochs')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.plot(train_loss, label='train')
  plt.plot(val_loss, label='val')
  plt.grid(True, which='both', axis='both')
  plt.tight_layout()
  plt.legend()
  plt.show()


def train_model(model, dataloaders, criterion, optimizer, save_name, num_epochs=25,  is_inception=False):
  since = time.time()
  
  # informações para plotagem
  train_losses , train_accuracy = [],[]
  val_losses , val_accuracy = [],[]

  # Gerencia a Learning Rate
  scheduler = StepLR(optimizer, step_size=3, gamma=0.5)

  val_acc_history = []

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
 

  for epoch in range(1, num_epochs+1):
    print('Epoch {}/{}'.format(epoch, num_epochs))
    print('-' * 10)

    epochStart = time.time()  

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
            
      if phase == 'train':
          model.train()  # Set model to training mode
      else:
          model.eval()   # Set model to evaluate mode

      running_loss = 0.0
      running_corrects = 0
      epoch_loss = 0.0
      epoch_acc = 0.0

      # Iterate over data.
      for inputs, labels in dataloaders[phase]:

        if phase == 'train':
          for i in range(len(inputs)): 
            noise = randint(0,2)
            # Se for 0 não aplica ruído à imagem
            if noise == 1:
              # aplica ruído gaussiano à imagem
              inputs[i] = torch.tensor(random_noise(inputs[i], mode='gaussian', mean=0, var=0.05, clip=True))
            if noise == 2:
              # aplica speckle noise à imagem
              inputs[i] = torch.tensor(random_noise(inputs[i], mode='speckle', mean=0, var=0.05, clip=True))


        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'train'):
          # Get model outputs and calculate loss
          # Special case for inception because in training it has an auxiliary output. In train
          #   mode we calculate the loss by summing the final output and the auxiliary output
          #   but in testing we only consider the final output.
          if is_inception and phase == 'train':
            # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
            outputs, aux_outputs = model(inputs)
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux_outputs, labels)
            loss = loss1 + 0.4*loss2
          else:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

          _, preds = torch.max(outputs, 1)

          # backward + optimize only if in training phase
          if phase == 'train':
            loss.backward()
            optimizer.step()

          # statistics
          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == labels.data)
        
      epoch_loss = running_loss / len(dataloaders[phase].dataset)
      epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)      
      
      if phase == 'train':
        train_losses.append(epoch_loss)
        train_accuracy.append(epoch_acc.item())         
      else:
        val_losses.append(epoch_loss)
        val_accuracy.append(epoch_acc.item())    

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
      

      # deep copy the model
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        salvar_modelo(model, '/content/drive/My Drive/Modelos Salvos/'+save_name+'.acc')
        print('Val acc up saving model')
      if phase == 'val':
        val_acc_history.append(epoch_acc)    
    scheduler.step()
    epochEnd = time.time()
    print('Duração:', (epochEnd - epochStart))
  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))
  
  #plota os gráficos
  plotar(train_losses, train_accuracy, val_losses, val_accuracy)

  #salvar dados da plotagem
  train = {'loss': train_losses, 'acc': train_accuracy}
  valid = {'loss': val_losses, 'acc': val_accuracy}

  df = pd.DataFrame(data=train)
  df.to_csv('/content/drive/My Drive/Modelos Salvos/'+save_name+'_train.csv')

  df = pd.DataFrame(data=valid)
  df.to_csv('/content/drive/My Drive/Modelos Salvos/'+save_name+'_valid.csv')
  
  # load best model weights
  model.load_state_dict(best_model_wts)
  return model, val_acc_history


def set_parameter_requires_grad(model, feature_extracting):
  if feature_extracting:
    for param in model.parameters():
      param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
  # Initialize these variables which will be set in this if statement. Each of these
  
  #   variables is model specific.
  model_ft = None
  input_size = 0

  if model_name == "resnet":
    """ Resnet18
    """
    model_ft = models.resnet18(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    set_parameter_requires_grad(model_ft.layer4, True)
    
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224

  elif model_name == "alexnet":
    """ Alexnet
    """
    model_ft = models.alexnet(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft.features, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224

  elif model_name == "vgg":
    """ VGG11_bn
    """
    model_ft = models.vgg19_bn(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224

  elif model_name == "squeezenet":
    """ Squeezenet
    """
    model_ft = models.squeezenet1_0(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft.features, feature_extract)
    model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
    model_ft.num_classes = num_classes
    input_size = 224

  elif model_name == "densenet":
    """ Densenet
    """
    model_ft = models.densenet121(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    input_size = 224

  elif model_name == "inception":
    """ Inception v3
    Be careful, expects (299,299) sized images and has auxiliary output
    """
    model_ft = models.inception_v3(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    # Handle the auxilary net
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
    # Handle the primary net
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs,num_classes)
    input_size = 299

  elif model_name == "googlenet":
    """ GoogLeNet
    """
    model_ft = models.googlenet(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224

  else:
      print("Invalid model name, exiting...")
      exit()

  return model_ft, input_size




# Resnet


In [ ]:
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

for param in model_ft.layer4.parameters():
  param.requires_grad = True

loaders = prepararDados(input_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = model_ft.to(device)

# lista dos parãmetros que serão aprendidos
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.001, weight_decay=0.000125)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train and evaluate
model_ft, hist = train_model(model_ft, loaders, criterion, optimizer_ft, model_name, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
resnet, input_size = carregar_resnet()
loaders = prepararDados(input_size)
print('Train set')
metricasModelo(resnet, loaders['train'])
print('Test set')
metricasModelo(resnet, loaders['test'])

# Inception V3

In [ ]:
model_name = "inception"

# Number of classes in the dataset
num_classes = 2

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
for param in model_ft.Mixed_7c.parameters():
  param.requires_grad = True

loaders = prepararDados(input_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = model_ft.to(device)

# lista dos parãmetros que serão aprendidos
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.001, weight_decay=0.000125)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train and evaluate
model_ft, hist = train_model(model_ft, loaders, criterion, optimizer_ft, model_name, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
inception, input_size = carregar_inception()
loaders = prepararDados(input_size)
print('Train set')
metricasModelo(inception, loaders['train'])
print('Test set')
metricasModelo(inception, loaders['test'])

# DenseNet


In [ ]:
model_name = "densenet"

# Number of classes in the dataset
num_classes = 2

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

for param in model_ft.features.denseblock4.denselayer15.parameters():
  param.requires_grad = True

for param in model_ft.features.denseblock4.denselayer16.parameters():
  param.requires_grad = True

loaders = prepararDados(input_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = model_ft.to(device)

# lista dos parãmetros que serão aprendidos
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.001, weight_decay=0.000125)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train and evaluate
model_ft, hist = train_model(model_ft, loaders, criterion, optimizer_ft, model_name, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
densenet, input_size = carregar_densenet()
loaders = prepararDados(input_size)
print('Train set')
metricasModelo(densenet, loaders['train'])
print('Test set')
metricasModelo(densenet, loaders['test'])

# AlexNet

In [ ]:
model_name = "alexnet"

# Number of classes in the dataset
num_classes = 2

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

loaders = prepararDados(input_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = model_ft.to(device)

# lista dos parãmetros que serão aprendidos
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.001, weight_decay=0.000125)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train and evaluate
model_ft, hist = train_model(model_ft, loaders, criterion, optimizer_ft, model_name, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
alexnet, input_size = carregar_alexnet()
loaders = prepararDados(input_size)
print('Train set')
metricasModelo(alexnet, loaders['train'])
print('Test set')
metricasModelo(alexnet, loaders['test'])

# GoogLeNet

In [ ]:
model_name = "googlenet"

# Number of classes in the dataset
num_classes = 2

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

model_ft = models.googlenet(pretrained=feature_extract)
set_parameter_requires_grad(model_ft, feature_extract)
for param in model_ft.inception5b.parameters():
  param.requires_grad = True
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, num_classes)
input_size = 224

loaders = prepararDados(input_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = model_ft.to(device)

# lista dos parãmetros que serão aprendidos
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.001, weight_decay=0.000125)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()




In [ ]:
# Train and evaluate
model_ft, hist = train_model(model_ft, loaders, criterion, optimizer_ft, model_name, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
googlenet, input_size = carregar_googlenet()
loaders = prepararDados(input_size)
print('Train set')
metricasModelo(model_ft, loaders['train'])
print('Test set')
metricasModelo(model_ft, loaders['test'])

# VGG

In [ ]:
model_name = 'vgg'

num_epochs = 50

model_ft = models.vgg19_bn(pretrained=True)
for param in model_ft.features.parameters():
  param.requires_grad = False
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 2)
input_size = 224

loaders = prepararDados(input_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = model_ft.to(device)

# lista dos parãmetros que serão aprendidos
params_to_update = model_ft.parameters()
print("Params to learn:")
if True:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.001, weight_decay=0.000125)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train and evaluate
model_ft, hist = train_model(model_ft, loaders, criterion, optimizer_ft, model_name, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
vgg, input_size = carregar_vgg()
loaders = prepararDados(input_size)
print('Test set')
metricasModelo(vgg, loaders['test'])

# Agrupamento de redes

In [ ]:
#load the models
resnet, input_size = carregar_resnet()
vgg, input_size = carregar_vgg()
densenet, input_size = carregar_densenet()
alexnet, input_size = carregar_alexnet()
googlenet, input_size = carregar_googlenet()


In [ ]:
#evaluate the models
loaders = prepararDados(input_size)

ini = time.time()
print('VGG19 - Test set')
metricasModelo(vgg, loaders['test'])
print(time.time() - ini);

ini = time.time()
print('Resnet18 - Test set')
metricasModelo(resnet, loaders['test'])
print(time.time() - ini);

ini = time.time()
print('Densenet121 - Test set')
metricasModelo(densenet, loaders['test'])
print(time.time() - ini);

ini = time.time()
print('Alexnet - Test set')
metricasModelo(alexnet, loaders['test'])
print(time.time() - ini);

ini = time.time()
print('GoogleNet - Test set')
metricasModelo(googlenet, loaders['test'])
print(time.time() - ini);


In [ ]:
# Usando predição por votação 
loaders = prepararDados(input_size)
print('========= Métricas do modelo ============\n')
ini = time.time()
y_pred, y_test = testeAgrupamentoMediaLogits(resnet, vgg, densenet, alexnet, googlenet, loaders['test'])
print(time.time() - ini)
target_names = ['normal', 'pneumonia']
print(classification_report(y_test, y_pred, target_names=target_names, digits=3))
print('\n========= Confusion Matrix ============\n')
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[0,1])
fig, ax = plt.subplots(figsize=(3,3))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300)
plt.show()

In [ ]:
# teste agrupamento usando votação

print('========= Métricas do modelo ============\n')
ini = time.time()
y_pred, y_test = testeAgrupamento(resnet, vgg, densenet, alexnet, googlenet, loaders['test'])
print(time.time() - ini)
target_names = ['normal', 'pneumonia']
print(classification_report(y_test, y_pred, target_names=target_names, digits=3))
print('\n========= Confusion Matrix ============\n')
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[0,1])
fig, ax = plt.subplots(figsize=(3,3))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300)
plt.show()

Teste dos modelos podados

In [ ]:
#evaluate the models
loaders = prepararDados(input_size)

ini = time.time()
print('VGG19 - Test set')
metricasModelo(vgg, loaders['test'])
print(time.time() - ini);

ini = time.time()
print('Resnet18 - Test set')
metricasModelo(resnet, loaders['test'])
print(time.time() - ini);

ini = time.time()
print('Densenet121 - Test set')
metricasModelo(densenet, loaders['test'])
print(time.time() - ini);

ini = time.time()
print('Alexnet - Test set')
metricasModelo(alexnet, loaders['test'])
print(time.time() - ini);

ini = time.time()
print('GoogleNet - Test set')
metricasModelo(googlenet, loaders['test'])
print(time.time() - ini);


In [ ]:
# Usando predição por votação 
loaders = prepararDados(input_size)
print('========= Métricas do modelo ============\n')
ini = time.time()
y_pred, y_test = testeAgrupamento(resnet, vgg, densenet, alexnet, googlenet, loaders['test'])
print(time.time() - ini)
target_names = ['normal', 'pneumonia']
print(classification_report(y_test, y_pred, target_names=target_names, digits=3))
print('\n========= Confusion Matrix ============\n')
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[0,1])
fig, ax = plt.subplots(figsize=(3,3))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300)
plt.show()

In [ ]:
# teste agrupamento usando votação

print('========= Métricas do modelo ============\n')
ini = time.time()
y_pred, y_test = testeAgrupamento(resnet, vgg, densenet, alexnet, googlenet, loaders['test'])
print(time.time() - ini)
target_names = ['normal', 'pneumonia']
print(classification_report(y_test, y_pred, target_names=target_names, digits=3))
print('\n========= Confusion Matrix ============\n')
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[0,1])
fig, ax = plt.subplots(figsize=(3,3))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
plt.xlabel('predicted label')
plt.ylabel('true label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300)
plt.show()

# Acesso dados


In [ ]:
#preparação dos dados
loaders = prepararDados(224)
train_loader = loaders['train']
batch = next(iter(train_loader))
images, labels = batch

print(labels)

In [ ]:
# Acessando um batch de elementos do dataset
batch = next(iter(train_loader))
images, labels = batch
print(images.shape)
print(images[0])
grid = torchvision.utils.make_grid(images, nrow=8)
plt.figure(figsize=(20,20))
plt.imshow(np.transpose(grid, (1,2,0)))
print('Labels: ', labels)
plt.show()

In [ ]:
# Acessando um batch de elementos do dataset
batch = next(iter(train_loader))
images, labels = batch

for i in range(len(images)): 
  noise = randint(0,2)
  if noise == 1:
    #aplica ruído gaussiano à imagem
    images[i] = torch.tensor(random_noise(images[i], mode='gaussian', mean=0, var=0.05, clip=True))
  if noise == 2:
    #aplica speckle noise à imagem
    images[i] = torch.tensor(random_noise(images[i], mode='speckle', mean=0, var=0.05, clip=True))


print(images.shape)
grid = torchvision.utils.make_grid(images, nrow=8)
plt.figure(figsize=(20,20))
plt.imshow(np.transpose(grid, (1,2,0)))
print('Labels: ', labels)
plt.show()

# Modelo Estudante


In [ ]:
class Network(nn.Module):
    def __init__(self):
        """CNN Builder."""
        super().__init__()

        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=24, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
           
        )

        self.fc_layer = nn.Sequential(
           
            nn.Linear(14*14*24, 2),
       
         
        )

    def forward(self, x):  

        # conv layers
        x = self.conv_layer(x)

        # flatten
        x = x.view(x.size(0), -1)

        # fc layer
        x = self.fc_layer(x)

        return x



In [ ]:
model_name = 'aluno_50'

num_epochs = 50

aluno = Network()

loaders = prepararDados(224)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

aluno = aluno.to(device)

# Observe that all parameters are being optimized
optimizer = optim.Adam(aluno.parameters(), lr=0.001, weight_decay=0.000125)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

print(aluno)

In [ ]:
# Train and evaluate
aluno, hist = train_model(aluno, loaders, criterion, optimizer, model_name, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
aluno = carregar_cnn('/content/drive/My Drive/Modelos Salvos/aluno_50.acc')
print('Modelo estudante - Test set')
ini = time.time()
metricasModelo(aluno, loaders['test'])
print(time.time() - ini)

In [ ]:
model_name = 'aluno_kd_v50'

ensemble = carregar_ensemble()

num_epochs = 50

aluno = Network()

loaders = prepararDados(224)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

aluno = aluno.to(device)

# Observe that all parameters are being optimized
optimizer = optim.Adam(aluno.parameters(), lr=0.001, weight_decay=0.000125)

T = 2
alpha = 0.9

# T=2 alpha = 0.9
aluno = train_model_kd_ensemble(aluno, ensemble, loaders, optimizer, model_name, T, alpha, num_epochs)

In [ ]:
print('Modelo estudante - Train set')
metricasModelo(aluno, loaders['train'])
print('Modelo estudante - Test set')
metricasModelo(aluno, loaders['test'])

# Treinamento KD


In [ ]:
def train_model_kd(model, teacher, dataloaders, optimizer, save_name, T, alpha, num_epochs=25):
  since = time.time()
  
  # informações para plotagem
  train_losses , train_accuracy = [],[]
  val_losses , val_accuracy = [],[]

  # Gerencia a Learning Rate
  scheduler = StepLR(optimizer, step_size=3, gamma=0.5)

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0  

  for epoch in range(1, num_epochs+1):
    print('Epoch {}/{}'.format(epoch, num_epochs))
    print('-' * 10)

    epochStart = time.time() 
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
            
      if phase == 'train':
          model.train()  # Set model to training mode
      else:
          model.eval()   # Set model to evaluate mode

      running_loss = 0.0
      running_corrects = 0
      epoch_loss = 0.0
      epoch_acc = 0.0

      # Iterate over data.
      for inputs, labels in dataloaders[phase]:    
        
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'train'):          
          outputs = model(inputs)          
          
          if phase == 'train':
            teacher_outputs = teacher(inputs)
            loss = loss_fn_kd(outputs, labels, teacher_outputs, T, alpha)
            loss.backward()
            optimizer.step()
          else: 
            loss = F.cross_entropy(outputs, labels)      
          
          _, preds = torch.max(outputs, 1)
         
          # statistics
          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == labels.data)
        
      epoch_loss = running_loss / len(dataloaders[phase].dataset)
      epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)      
      
      if phase == 'train':
        train_losses.append(epoch_loss)
        train_accuracy.append(epoch_acc.item())         
      else:
        val_losses.append(epoch_loss)
        val_accuracy.append(epoch_acc.item())    

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))      

      # deep copy the model
      if phase == 'val' and epoch_acc >= best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        salvar_modelo(model, '/content/drive/My Drive/Modelos Salvos/'+save_name+'.acc')
        print('Val acc up --> saving model')
        
    scheduler.step()
    epochEnd = time.time()
    print('Duração:', (epochEnd - epochStart))
  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))
  
  #plota os gráficos
  plotar(train_losses, train_accuracy, val_losses, val_accuracy)

  #salva dados da plotagem
  train = {'loss': train_losses, 'acc': train_accuracy}
  valid = {'loss': val_losses, 'acc': val_accuracy}

  df = pd.DataFrame(data=train)
  df.to_csv('/content/drive/My Drive/Modelos Salvos/'+save_name+'_train.csv')

  df = pd.DataFrame(data=valid)
  df.to_csv('/content/drive/My Drive/Modelos Salvos/'+save_name+'_valid.csv')
  
  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

def loss_fn_kd(outputs, labels, teacher_outputs, T, alpha): 

  return nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1), F.softmax(teacher_outputs/T, dim=1)) * (alpha * T * T) + F.cross_entropy(outputs, labels) * (1. - alpha)

def train_model_kd_ensemble(model, ensemble, dataloaders, optimizer, save_name, T, alpha, num_epochs=25):
  since = time.time()

  if (torch.cuda.is_available()):
    resnet = ensemble['resnet']
    resnet = resnet.cuda()
    vgg = ensemble['vgg']
    vgg = vgg.cuda()
    densenet = ensemble['densenet']
    densenet = densenet.cuda()
    alexnet = ensemble['alexnet']
    alexnet = alexnet.cuda()
    googlenet = ensemble['googlenet']
    googlenet = googlenet.cuda()
  
  resnet.eval()
  densenet.eval()
  vgg.eval()
  alexnet.eval()
  googlenet.eval()

  
  # informações para plotagem
  train_losses , train_accuracy = [],[]
  val_losses , val_accuracy = [],[]

  # Gerencia a Learning Rate
  scheduler = StepLR(optimizer, step_size=3, gamma=0.5)

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0  

  for epoch in range(1, num_epochs+1):
    print('Epoch {}/{}'.format(epoch, num_epochs))
    print('-' * 10)

    epochStart = time.time() 
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
            
      if phase == 'train':
          model.train()  # Set model to training mode
      else:
          model.eval()   # Set model to evaluate mode

      running_loss = 0.0
      running_corrects = 0
      epoch_loss = 0.0
      epoch_acc = 0.0

      # Iterate over data.
      for inputs, labels in dataloaders[phase]:    
        
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'train'):          
          outputs = model(inputs)          
          
          if phase == 'train':
            output_resnet = resnet(inputs)
            output_densenet = densenet(inputs)
            output_alexnet = alexnet(inputs)
            output_googlenet = googlenet(inputs)
            output_vgg = vgg(inputs)

            teacher_outputs = (output_resnet+output_densenet+output_alexnet+output_googlenet+output_vgg)/5
            
            loss = loss_fn_kd(outputs, labels, teacher_outputs, T, alpha)
            loss.backward()
            optimizer.step()
          else: 
            loss = F.cross_entropy(outputs, labels)      
          
          _, preds = torch.max(outputs, 1)
         
          # statistics
          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == labels.data)
        
      epoch_loss = running_loss / len(dataloaders[phase].dataset)
      epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)      
      
      if phase == 'train':
        train_losses.append(epoch_loss)
        train_accuracy.append(epoch_acc.item())         
      else:
        val_losses.append(epoch_loss)
        val_accuracy.append(epoch_acc.item())    

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))      

      # deep copy the model
      if phase == 'val' and epoch_acc >= best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        salvar_modelo(model, '/content/drive/My Drive/Modelos Salvos/'+save_name+'.acc')
        print('Val acc up --> saving model')
        
    scheduler.step()
    epochEnd = time.time()
    print('Duração:', (epochEnd - epochStart))
  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))
  
  #plota os gráficos
  plotar(train_losses, train_accuracy, val_losses, val_accuracy)

  #salva dados da plotagem
  train = {'loss': train_losses, 'acc': train_accuracy}
  valid = {'loss': val_losses, 'acc': val_accuracy}

  df = pd.DataFrame(data=train)
  df.to_csv('/content/drive/My Drive/Modelos Salvos/'+save_name+'_train.csv')

  df = pd.DataFrame(data=valid)
  df.to_csv('/content/drive/My Drive/Modelos Salvos/'+save_name+'_valid.csv')
  
  # load best model weights
  model.load_state_dict(best_model_wts)
  return model


# Parâmetros



In [ ]:
def infoModelo(model):
  pytorch_total_params = 0
  for p in model.parameters(): 
    pytorch_total_params += p.numel()

  pytorch_trainable_params = 0
  for p in model.parameters(): 
    if p.requires_grad:
      pytorch_trainable_params += p.numel()
 
  print('Parâmetros treináveis: ', pytorch_trainable_params)
  print('Parâmetros não treináveis: ', pytorch_total_params-pytorch_trainable_params)
  print('Total: ', pytorch_total_params)

In [ ]:
vgg = models.vgg19_bn(pretrained=True)
for param in vgg.features.parameters():
  param.requires_grad = False
num_ftrs = vgg.classifier[6].in_features
vgg.classifier[6] = nn.Linear(num_ftrs, 2)
infoModelo(vgg)

In [ ]:
resnet, input_size = initialize_model('resnet', 2, True, use_pretrained=True)
for param in resnet.layer4.parameters():
  param.requires_grad = True
infoModelo(resnet)

In [ ]:
densenet, input_size = initialize_model('densenet', 2, True, use_pretrained=True)
for param in densenet.features.denseblock4.denselayer15.parameters():
  param.requires_grad = True

for param in densenet.features.denseblock4.denselayer16.parameters():
  param.requires_grad = True
infoModelo(densenet)

In [ ]:
model_ft, input_size = initialize_model('inception', 2, True, use_pretrained=True)
for param in model_ft.Mixed_7c.parameters():
  param.requires_grad = True
infoModelo(model_ft)

In [ ]:
model_ft, input_size = initialize_model('alexnet', 2,True , use_pretrained=True)
infoModelo(model_ft)

In [ ]:
model_ft = models.googlenet(pretrained=True)
set_parameter_requires_grad(model_ft, True)
for param in model_ft.inception5b.parameters():
  param.requires_grad = True
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
infoModelo(model_ft)

In [ ]:
cnn = Network()
infoModelo(cnn)